##Run a RAG Agent  
This notebook demonstrates how to build a Chroma vector database and use it to enable an augmented LLM call.  
We'll start by installing the chromadb vector database, litellm model interface, and pypdf PDF reader software.

In [ ]:
%pip install chromadb pypdf litellm

Let's ignore warnings to keep the screen as clean as we can.

In [2]:
import warnings
warnings.filterwarnings('ignore')

We'll now import the chromadb library for the vector database and the pypdf library to enable reading pdf files. We'll also need the litellm library to connect to our model, and we'll load Google's userdata so we can retrieve our Gemini key.


In [3]:
import os, chromadb
from litellm import completion
from pypdf import PdfReader
from google.colab import userdata

We'll then set up a client handle and create an empty collection. We'll also load our API key and set the model as gemini.

In [4]:
client     = chromadb.Client()
collection = client.create_collection(name="handbook")

os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
model = "gemini/gemini-2.5-flash"

Let's make sure we have our GAIA_Quality_Manual uploaded into the Colab local files environment.  
We'll now read the pdf file in and we'll get a list of pages as "page" type objects. We'll iterate through the pages and extract the text, and add each page in text form to our vector database collection.

In [ ]:
reader = PdfReader("GAIA_Quality_Manual.pdf")

texts=[]
metas=[]
ideas=[]
for page in reader.pages:
    texts.append(page.extract_text())
    metas.append({"source":"page"+str(len(texts))})
    ideas.append("idea"+str(len(texts)))
    collection.add(
        documents = texts,
        metadatas = metas,
        ids       = ideas)

We'll set up the system instructions for the AI model that we'll call and the user prompt.

In [7]:
system_instructions = """
   You are a helpful business architect who can identify business requirements.  You analyze the provided Context
   and select requirements which exist in the Context in order to answer the user Question This is the only place
   the requirements exist so do not use your own knowledge to answer the question, use only the Context.
   Attributes are single word adjectives describing a requirement. Identify any key performance attributes
   related to the requirements and the list of single word attributes that relate to them.
   You always return a json string in the following form:
   { 'requirement':'requirement 1','attribute_list':['attribute 1','attribute 2',..],..}
   If you cannot find the answer in the Context then reply that you do not know.
   """

query = "What skills should Laboratory employees have?"

We now need to build our context. We'll get back a set of documents, iterate through them., and build a text-based context.

In [ ]:
results = collection.query(query_texts=[query],n_results=5)
docs = results["documents"]
context = ""
for doc in docs:
    context = context.join(doc[0])
print("Context length is "+str(len(context)))

We'll now build our LLM user prompt, call the model, and print the response. We'll set the LLM temperature to 0.5 for a fairly constrained response.

In [ ]:
prompt  = f"""
   Answer the question based on the Context.
   Context:
   {context}
   Question:
   {query}
   """
response = completion(
              model = model,
              messages = [
                {"role":"system","content":system_instructions},
                {"role":"user","content":prompt}
              ],
              temperature=0.5,
           )
print(response['choices'][0]['message']['content'])